# AI Model Gateway Demo - Deployment Notebook

This notebook orchestrates the full deployment of the AI Model Gateway infrastructure and agents.

## Steps:
1. Deploy `infra/hub` - API Management and Azure OpenAI resources
2. Deploy `infra/spoke` - AI Foundry project with Model Gateway connection
3. Generate `.env` file from Terraform outputs
4. Deploy and test the agent

## Configuration

In [ ]:
# Install required packages (skip if already installed)
%pip install -q "azure-ai-projects>=2.0.0b3" "azure-identity>=1.25.1"

In [ ]:
import json
import os
import subprocess

LOCATION = "swedencentral"
HUB_DIR = "infra/hub"
SPOKE_DIR = "infra/spoke"
os.environ["ARM_SUBSCRIPTION_ID"] = ""
os.environ["ARM_TENANT_ID"] = ""
os.environ["AZURE_TENANT_ID"] = ""

## Helper Functions

In [ ]:
def run_terraform(working_dir: str, command: list[str], capture_output: bool = False) -> subprocess.CompletedProcess:
    """Run a terraform command in the specified directory."""
    full_command = ["terraform"] + command
    print(f"📂 {working_dir}")
    print(f"🔧 Running: {' '.join(full_command)}")

    result = subprocess.run(
        full_command,
        cwd=working_dir,
        capture_output=capture_output,
        text=True
    )

    if result.returncode != 0 and not capture_output:
        raise Exception(f"Terraform command failed with exit code {result.returncode}")

    return result


def get_terraform_output(working_dir: str) -> dict:
    """Get terraform outputs as a dictionary."""
    result = run_terraform(working_dir, ["output", "-json"], capture_output=True)
    if result.returncode != 0:
        raise Exception(f"Failed to get outputs: {result.stderr}")

    outputs = json.loads(result.stdout)
    # Extract just the values
    return {k: v["value"] for k, v in outputs.items()}

## Step 1: Deploy Hub Infrastructure

This deploys:
- Azure API Management
- Azure OpenAI with model deployments
- Application Insights

In [ ]:
# Initialize Hub
run_terraform(HUB_DIR, ["init", "-upgrade"])

In [ ]:
# Apply Hub deployment
run_terraform(HUB_DIR, [
    "apply",
    "-auto-approve",
    f"-var=location={LOCATION}"
])

In [ ]:
# Get Hub outputs
hub_outputs = get_terraform_output(HUB_DIR)
print("\n✅ Hub outputs:")
for key, value in hub_outputs.items():
    if "key" in key.lower():
        print(f"   {key}: ***REDACTED***")
    else:
        print(f"   {key}: {value}")

## Step 2: Deploy Spoke Infrastructure

This deploys:
- AI Foundry Account and Project
- Model Gateway connection to APIM

In [ ]:
# Initialize Spoke
run_terraform(SPOKE_DIR, ["init", "-upgrade"])

In [ ]:
# Prepare spoke variables from hub outputs
model_gateway_var = json.dumps({
    "url": hub_outputs["azure_openai_endpoint"],
    "api_key": hub_outputs["apim_subscription_key"],
    "metadata": hub_outputs["model_gateway_metadata"]
})

# Apply Spoke deployment
run_terraform(SPOKE_DIR, [
    "apply",
    "-auto-approve",
    f"-var=resource_group_name={hub_outputs['resource_group_name']}",
    f"-var=location={LOCATION}",
    f"-var=model_gateway={model_gateway_var}"
])

In [ ]:
# Get Spoke outputs
spoke_outputs = get_terraform_output(SPOKE_DIR)
print("\n✅ Spoke outputs:")
for key, value in spoke_outputs.items():
    print(f"   {key}: {value}")

## Step 3: Generate .env File

Create the `.env` file required by `deploy_agent.py` and `test_agent.py`

In [ ]:
# Generate .env content
env_content = f"""# Auto-generated by deployment notebook - do not edit manually

# Azure Configuration
AZURE_SUBSCRIPTION_ID={os.environ['ARM_SUBSCRIPTION_ID']}
AZURE_RESOURCE_GROUP={hub_outputs['resource_group_name']}

# AI Foundry Configuration
AZURE_AI_ACCOUNT_NAME={spoke_outputs['cognitive_account_name']}
AZURE_AI_PROJECT_NAME={spoke_outputs['project_name']}
AZURE_AI_PROJECT_ENDPOINT={spoke_outputs['project_endpoint']}

# Agent Configuration
AGENT_NAME=APIMGatewayAgent
AGENT_MODEL=model-gateway/gpt-4o-mini
"""

# Write .env file
with open(".env", "w") as f:
    f.write(env_content)

print("✅ Generated .env file:")
print("-" * 40)
# Print without sensitive data
for line in env_content.split("\n"):
    if line and not line.startswith("#"):
        print(f"   {line}")

## Step 4: Deploy Agent

Run `deploy_agent.py` to create the agent in AI Foundry

In [ ]:
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition
from azure.identity import DefaultAzureCredential

# Agent configuration
AGENT_NAME = "APIMGatewayAgent"
AGENT_MODEL = "model-gateway/gpt-4o-mini"

print("🚀 Deploying Agent via AI Gateway")
print("=" * 60)

# Initialize the project client
print("\n📡 Connecting to Azure AI Foundry project...")
print(f"   Project: {spoke_outputs['project_name']}")
print(f"   Endpoint: {spoke_outputs['project_endpoint']}")

project_client = AIProjectClient(
    endpoint=spoke_outputs['project_endpoint'],
    credential=DefaultAzureCredential()
)
print("✅ Connected successfully")

# Create an agent using the AI Gateway connection
print("\n🤖 Creating agent with AI Gateway connection...")
print(f"   Model: {AGENT_MODEL}")
print("   (This routes through APIM to Azure OpenAI)")

with project_client:
    agent = project_client.agents.create_version(
        agent_name=AGENT_NAME,
        definition=PromptAgentDefinition(
            model=AGENT_MODEL,
            instructions="You are a helpful AI assistant deployed via Azure API Management. "
            "All your requests are routed through APIM for monitoring and governance.",
        ),
    )

print("\n" + "=" * 60)
print("🎉 Agent Deployment Complete!")
print("=" * 60)
print("\n📋 Agent Details:")
print(f"   Agent Name: {agent.name}")
print(f"   Agent Version: {agent.version}")
print(f"   Model: {AGENT_MODEL}")

## Step 5: Test Agent

Run `test_agent.py` to verify the agent works through the APIM gateway

In [ ]:
print("🧪 Testing APIM Gateway Prompt Agent")
print("=" * 60)

project_client = AIProjectClient(
    endpoint=spoke_outputs['project_endpoint'],
    credential=DefaultAzureCredential()
)

with project_client:
    print(f"\n📋 Using Agent: {agent.name} (version {agent.version})")
    print(f"   Model: {AGENT_MODEL}")
    print("   (Routes through APIM)")

    # Test message
    test_message = "Hello! Please tell me a short joke about AI and API gateways."
    print(f"\n💬 User: {test_message}")

    print("\n🔄 Processing through APIM Gateway...")

    # Use the OpenAI client and Conversations/Responses API
    with project_client.get_openai_client() as openai_client:
        # Create a conversation with initial user message
        conversation = openai_client.conversations.create(
            items=[{"type": "message", "role": "user", "content": test_message}],
        )
        print(f"   Conversation created: {conversation.id}")

        # Get response from agent
        response = openai_client.responses.create(
            conversation=conversation.id,
            extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
            input="",
        )

        print("\n🤖 Assistant Response:")
        print(f"   {response.output_text}")

print("\n" + "=" * 60)
print("✅ Test Complete!")
print("=" * 60)
print("\n📊 Request Flow:")
print("   1. Python SDK → Azure AI Foundry Project")
print("   2. Foundry → model-gateway connection")
print(f"   3. Connection → APIM ({hub_outputs['apim_gateway_url']})")
print("   4. APIM → Azure OpenAI (via managed identity)")
print("   5. Response flows back through APIM (with monitoring/governance)")

## Cleanup (Optional)

Destroy all infrastructure when done testing

In [ ]:
# ⚠️ DANGER: Uncomment to destroy all resources
# print("🗑️ Destroying Spoke...")
# run_terraform(SPOKE_DIR, ["destroy", "-auto-approve",
#     f"-var=resource_group_name={hub_outputs['resource_group_name']}",
#     f"-var=location={LOCATION}",
#     f"-var=model_gateway={model_gateway_var}"
# ])

# print("🗑️ Destroying Hub...")
# run_terraform(HUB_DIR, ["destroy", "-auto-approve",
#     f"-var=location={LOCATION}"
# ])

# print("✅ All resources destroyed")